<a href="https://colab.research.google.com/github/kkkonts/coursera-matstat-AB/blob/main/hw03_vanilla_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Пришло время самостоятельно написать парсер! Мы будем собирать данные [о ценах на книги.](http://books.toscrape.com)


> __ВНИМАНИЕ!__ Почему-то у Coursera после обновления на этом задании перестал работать грейдер. В связи с этим все ответы, которые вы получите при написании кода, придется перенести в тест, который идёт в курсе сразу после этого задания.

In [2]:
!pip3 install lxml
!pip install selenium
!pip3 install fake_useragent

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00


In [3]:
# Подгрузите все необходимые для работы пакеты.

# Если ваш код будет ругаться, что нет пакета lxml, установите его
# Для этого выполните в одной из ячеек команду !pip3 install lxml
from typing import List
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm_notebook
import time
import pandas as pd
import numpy as np
from fake_useragent import UserAgent


In [4]:
# фиктивная проверка

Прогуляйтесь на сайт http://books.toscrape.com/ и изучите его структуру.  


# 1. Сбор ссылок на книги

Напишите функцию `get_soup`, которая по ссылке возвращает html-разметку страницы в формате `bs4`

In [5]:
def get_page_soup(url_link):
    response = requests.get(page_link)
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    return soup

page_link = 'http://books.toscrape.com/'
get_page_soup(page_link).html.head.title

<title>
    All products | Books to Scrape - Sandbox
</title>

In [6]:
main_url = 'http://books.toscrape.com/catalogue/'
page_number = 5

soup = get_page_soup(main_url + f'page-{page_number}.html')
soup.find_all('a')[0].text

'Books to Scrape'

Напишите функцию `get_books_links`, которая находит в html-разметке страницы ссылки на странички с отдельными книгами.

In [7]:
def get_book_links(page_soup) -> List[str]:
    a_list = soup.select("h3 > a")
    #a = h3.find('a').attrs['href']

    return [a.attrs['href'] for a in a_list]
links = get_book_links(soup)
links

['catalogue/a-light-in-the-attic_1000/index.html',
 'catalogue/tipping-the-velvet_999/index.html',
 'catalogue/soumission_998/index.html',
 'catalogue/sharp-objects_997/index.html',
 'catalogue/sapiens-a-brief-history-of-humankind_996/index.html',
 'catalogue/the-requiem-red_995/index.html',
 'catalogue/the-dirty-little-secrets-of-getting-your-dream-job_994/index.html',
 'catalogue/the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html',
 'catalogue/the-boys-in-the-boat-nine-americans-and-their-epic-quest-for-gold-at-the-1936-berlin-olympics_992/index.html',
 'catalogue/the-black-maria_991/index.html',
 'catalogue/starving-hearts-triangular-trade-trilogy-1_990/index.html',
 'catalogue/shakespeares-sonnets_989/index.html',
 'catalogue/set-me-free_988/index.html',
 'catalogue/scott-pilgrims-precious-little-life-scott-pilgrim-1_987/index.html',
 'catalogue/rip-it-up-and-start-again_986/index.html',
 'catalogue/our-band-could-be-your-life-scene

С помощью цикла соберите в лист `book_links` первые 200 книг.

In [8]:
book_links = links
main_url = 'http://books.toscrape.com/catalogue/'
page_number = 2

while len(book_links) < 200:
    soup = get_page_soup(main_url + f'page-{page_number}.html')
    page_number += 1
    book_links.extend(get_book_links(soup))

len(book_links)

200

# 2. Сбор информации о книгах

Напишите несколько небольших функций, которые собирают различные данные об одной книге, необходимые для ответов на вопросы ниже. Информацию о книге собирайте в виде словаря вида

```
{ 'name': 'Преступление и наказание', 'rating': 1, 'description': 'ужасно депрессивная книга', ... }

```

In [33]:
let_to_num = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}

def get_book_info(book_link: str) -> dict:
    book_info = {}
    book_link = 'http://books.toscrape.com/' + book_link

    response = requests.get(book_link)
    html = html = response.content
    book_soup = BeautifulSoup(html,'html.parser')

    book_info['name'] = book_soup.find(class_='product_main').h1.text
    book_info['rating'] = let_to_num[book_soup.find(class_='star-rating').attrs['class'][1]]
    book_info['description'] = book_soup.find(lambda tag: tag.name == 'p' and not tag.attrs).text

    table = book_soup.find(class_='table')
    book_info['tax'] = float(table.find('th', text='Tax').parent.find('td').text[1:])
    book_info['price'] = float(table.find('th', text='Price (excl. tax)').parent.find('td').text[1:])


    return book_info

get_book_info(links[2])

<ipython-input-33-4b0cff82caa4>:16: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  book_info['tax'] = float(table.find('th', text='Tax').parent.find('td').text[1:])
<ipython-input-33-4b0cff82caa4>:17: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  book_info['price'] = float(table.find('th', text='Price (excl. tax)').parent.find('td').text[1:])


{'name': 'Soumission',
 'rating': 1,
 'description': 'Dans une France assez proche de la nôtre, un homme s’engage dans la carrière universitaire. Peu motivé par l’enseignement, il s’attend à une vie ennuyeuse mais calme, protégée des grands drames historiques. Cependant les forces en jeu dans le pays ont fissuré le système politique jusqu’à provoquer son effondrement. Cette implosion sans soubresauts, sans vraie révolution, s Dans une France assez proche de la nôtre, un homme s’engage dans la carrière universitaire. Peu motivé par l’enseignement, il s’attend à une vie ennuyeuse mais calme, protégée des grands drames historiques. Cependant les forces en jeu dans le pays ont fissuré le système politique jusqu’à provoquer son effondrement. Cette implosion sans soubresauts, sans vraie révolution, se développe comme un mauvais rêve.Le talent de l’auteur, sa force visionnaire nous entraînent sur un terrain ambigu et glissant ; son regard sur notre civilisation vieillissante fait coexister da

Пройдите циклом по всем сыслкам из списка `book_links` и соберите данные о книгах в вектор `book_info`.

In [41]:
book_info = []

for link in tqdm_notebook(links):
    book_info.append(get_book_info(link))

book_info[-3:]

<ipython-input-41-2a0d0edd392f>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for link in tqdm_notebook(links):


  0%|          | 0/200 [00:00<?, ?it/s]

<ipython-input-33-4b0cff82caa4>:16: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  book_info['tax'] = float(table.find('th', text='Tax').parent.find('td').text[1:])
<ipython-input-33-4b0cff82caa4>:17: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  book_info['price'] = float(table.find('th', text='Price (excl. tax)').parent.find('td').text[1:])


[{'name': 'Mesaerion: The Best Science Fiction Stories 1800-1849',
  'rating': 1,
  'description': 'Andrew Barger, award-winning author and engineer, has extensively researched forgotten journals and magazines of the early 19th century to locate groundbreaking science fiction short stories in the English language. In doing so, he found what is possibly the first science fiction story by a female (and it is not from Mary Shelley). Andrew located the first steampunk short Andrew Barger, award-winning author and engineer, has extensively researched forgotten journals and magazines of the early 19th century to locate groundbreaking science fiction short stories in the English language. In doing so, he found what is possibly the first science fiction story by a female (and it is not from Mary Shelley). Andrew located the first steampunk short story, which has not been republished since 1844. There is the first voyage to the moon in a balloon, republished for the first time since 1820 that f

Превратим вектор из информации в полноценную таблицу с данными.

In [37]:
book_info_df = pd.DataFrame(book_info)
print(book_info_df.shape)
book_info_df.head(10)

(200, 5)


,name,rating,description,tax,price
0,A Light in the Attic,3,It's hard to imagine a world without A Light i...,0.0,51.77
1,Tipping the Velvet,1,"""Erotic and absorbing...Written with starling ...",0.0,53.74
2,Soumission,1,"Dans une France assez proche de la nôtre, un h...",0.0,50.10
3,Sharp Objects,4,"WICKED above her hipbone, GIRL across her hear...",0.0,47.82
4,Sapiens: A Brief History of Humankind,5,From a renowned historian comes a groundbreaki...,0.0,54.23
5,The Requiem Red,1,Patient Twenty-nine.A monster roams the halls ...,0.0,22.65
6,The Dirty Little Secrets of Getting Your Dream...,4,Drawing on his extensive experience evaluating...,0.0,33.34
7,The Coming Woman: A Novel Based on the Life of...,3,"""If you have a heart, if you have a soul, Kare...",0.0,17.93
8,The Boys in the Boat: Nine Americans and Their...,4,For readers of Laura Hillenbrand's Seabiscuit ...,0.0,22.60
9,The Black Maria,1,"Praise for Aracelis Girmay:""[Girmay's] every l...",0.0,52.15


Теперь, когда все данные собраны, настало время ответить на несколько вопросов:

- У скольких книг отсутствует описание? Ответ на этот вопрос вбейте в тест, который идёт после лабораторной работы.

In [29]:
ans1 = book_info_df['description'].isna().sum()
ans1

0

- Сколько раз в данных встречается налог, больший нуля?  Ответ на этот вопрос вбейте в тест, который идёт после лабораторной работы.

In [38]:
ans2 = book_info_df[book_info_df['tax'] > 0].shape[0]
ans2

0

- Сколько раз рейтинг книги составлял пять звезд?  Ответ на этот вопрос вбейте в тест, который идёт после лабораторной работы.

In [31]:
ans3 = book_info_df[book_info_df['rating'] == 5].shape[0]
ans3

40

- Какова средняя цена книг (без учета налога)?  Ответ на этот вопрос вбейте в тест, который идёт после лабораторной работы.

In [40]:
ans4 = book_info_df['price'].mean()
ans4

38.0485